# Fuzzy Matching Attempt

## Description
This is my first attempt at a proper solution to this problem.  To deal with inevitable spelling mistakes and missing words I'll use fuzzy text matching.  This is an inprecise method of text matching that produces a score based on how strong the match is.  Unless a direct text match is found, we cannot be sure of the match, so we can use some confidence level to determine if the match is close enough.

In [561]:

import numpy as np
import pandas as pd
import re
from fuzzywuzzy import fuzz
import nltk
from nltk.corpus import stopwords
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 200)

In [562]:
content_titles_df = pd.read_csv('ProvidedFiles/Content sample.csv')
content_titles_df['transformed'] = content_titles_df.apply(lambda x: re.sub(' +', ' ',re.sub(r'[^\w]', ' ', x['Content_name']).lower()), axis=1)
survey_response_df = pd.read_csv('ProvidedFiles/Survey response sample data.csv')
survey_response_df['transformed'] = survey_response_df.apply(lambda x: re.sub(' +', ' ',re.sub(r'[^\w]', ' ', x['Response']).lower()), axis=1)

In [563]:
survey_response_df.head()

,Customer_id,Response,transformed
0,1,"Fear the walking dead,Supernatural (huge fan and sad it has finished),The Gentlemen, Outlander",fear the walking dead supernatural huge fan and sad it has finished the gentlemen outlander
1,2,A lot!\n\n-good doctor\n-gangs of London \n- the gentleman\n-ma \n-spies in disguise,a lot good doctor gangs of london the gentleman ma spies in disguise
2,3,"Miss scarlet and the duke,knifes out,Dublin murders",miss scarlet and the duke knifes out dublin murders
3,4,"History drama-Vikings,Kid friendly-Casper,Sometimes the conversations while watching Neon can get serious but we all end up having fun together, :)",history drama vikings kid friendly casper sometimes the conversations while watching neon can get serious but we all end up having fun together
4,5,"The Undoing,Game of thrones,Outlander, Vikings,CB Strike (and most all British dramas) Westworld",the undoing game of thrones outlander vikings cb strike and most all british dramas westworld


In [564]:
customer_id_column = []
response_column = []
title_column = []
score_column = []
for index_2, row_2 in survey_response_df.iterrows():
    
    for index_1, row_1 in content_titles_df.iterrows():
        customer_id_column.append(row_2['Customer_id'])
        response_column.append(row_2['Response'])
        title_column.append(row_1['Content_name'])
        score_column.append(fuzz.partial_ratio(row_1['transformed'], row_2['transformed']))
result_dict = {
    'Customer_id':customer_id_column,
    'Response':response_column,
    'Title':title_column,
    'Score': score_column}
result_df = pd.DataFrame(result_dict).sort_values(by=['Score'], ascending=False)
result_df.head(10)

,Customer_id,Response,Title,Score
0,1,"Fear the walking dead,Supernatural (huge fan and sad it has finished),The Gentlemen, Outlander",Fear the Walking Dead,100
22,2,A lot!\n\n-good doctor\n-gangs of London \n- the gentleman\n-ma \n-spies in disguise,Gangs of London,100
82,5,"The Undoing,Game of thrones,Outlander, Vikings,CB Strike (and most all British dramas) Westworld",Game of Thrones,100
81,5,"The Undoing,Game of thrones,Outlander, Vikings,CB Strike (and most all British dramas) Westworld",The Undoing,100
79,5,"The Undoing,Game of thrones,Outlander, Vikings,CB Strike (and most all British dramas) Westworld",Vikings,100
74,5,"The Undoing,Game of thrones,Outlander, Vikings,CB Strike (and most all British dramas) Westworld",Ma,100
71,5,"The Undoing,Game of thrones,Outlander, Vikings,CB Strike (and most all British dramas) Westworld",Outlander,100
63,4,"History drama-Vikings,Kid friendly-Casper,Sometimes the conversations while watching Neon can get serious but we all end up having fun together, :)",Casper,100
62,4,"History drama-Vikings,Kid friendly-Casper,Sometimes the conversations while watching Neon can get serious but we all end up having fun together, :)",Vikings,100
57,4,"History drama-Vikings,Kid friendly-Casper,Sometimes the conversations while watching Neon can get serious but we all end up having fun together, :)",Ma,100


## Addressing some problems
The major issue with this fuzzy matching library is it's lack of white space consideration.  The mention of the title 'Marley and Me' would produce a partial ratio match with the title 'Ma'.  Partial matching is still important in the context of whole words however, so what we want is a function which fuzzy matches sub strings within the survey response that are only sliced along white space boundaries, that exceed or equal the length of the title we are searching for.

## Implementation:

In [565]:
def find_next_space(start_index : int, minimum_search_distance : int, text : str):
    check_index = start_index+minimum_search_distance
    space_index = -1
    alphas_observed = 0
    for i in range(start_index, len(text)):
        if text[i] != ' ':
            alphas_observed+=1
        elif alphas_observed>=minimum_search_distance:
            space_index = i
            break
    #space_index = text[start_index+minimum_search_distance:].find(' ')
    if space_index == -1:
        return len(text)
    else:
        return space_index

def my_fuzzy_match(content_title, survey_response : str) -> int:
    substring_left_index = 0
    title_len = len(content_title.replace(' ' , ''))
    substring_right_index = find_next_space(substring_left_index, title_len, survey_response)
    substring = survey_response[substring_left_index : substring_right_index]
    highest_score = 0
    while True:
        new_score = fuzz.ratio(substring.replace(' ',''), content_title.replace(' ',''))
        highest_score = max(highest_score, new_score)
        if highest_score == 100:
            break
        substring_left_index = find_next_space(substring_left_index, 1, survey_response)+1
        if substring_left_index >= len(survey_response)-1:
            break
        substring_right_index = find_next_space(substring_left_index, title_len, survey_response)
        substring = survey_response[substring_left_index : substring_right_index]
    return highest_score
    

## Proof of concept:

In [566]:
survey_example = 'I really enjoyed the movie Marley and Me'
title_1 = 'Marley and Me'
title_2 = 'Ma'
print("Old method:")
print(fuzz.partial_ratio(title_1.lower(), survey_example.lower()))
print(fuzz.partial_ratio(title_2.lower(), survey_example.lower()))
print()
print("New method:")
print(my_fuzzy_match(title_1.lower(), survey_example.lower()))
print(my_fuzzy_match(title_2.lower(), survey_example.lower()))

Old method:
100
100

New method:
100
50


In [567]:
customer_id_column = []
response_column = []
title_column = []
score_column = []
for index_2, row_2 in survey_response_df.iterrows():
    
    for index_1, row_1 in content_titles_df.iterrows():
        customer_id_column.append(row_2['Customer_id'])
        response_column.append(row_2['Response'])
        title_column.append(row_1['Content_name'])
        score_column.append(my_fuzzy_match(row_1['transformed'], row_2['transformed']))
result_dict = {
    'Customer_id':customer_id_column,
    'Response':response_column,
    'Title':title_column,
    'Score': score_column}
result_df = pd.DataFrame(result_dict).sort_values(by=['Score'], ascending=False)
result_df.head(10)

,Customer_id,Response,Title,Score
0,1,"Fear the walking dead,Supernatural (huge fan and sad it has finished),The Gentlemen, Outlander",Fear the Walking Dead,100
22,2,A lot!\n\n-good doctor\n-gangs of London \n- the gentleman\n-ma \n-spies in disguise,Gangs of London,100
83,5,"The Undoing,Game of thrones,Outlander, Vikings,CB Strike (and most all British dramas) Westworld",C.B. Strike,100
82,5,"The Undoing,Game of thrones,Outlander, Vikings,CB Strike (and most all British dramas) Westworld",Game of Thrones,100
81,5,"The Undoing,Game of thrones,Outlander, Vikings,CB Strike (and most all British dramas) Westworld",The Undoing,100
79,5,"The Undoing,Game of thrones,Outlander, Vikings,CB Strike (and most all British dramas) Westworld",Vikings,100
71,5,"The Undoing,Game of thrones,Outlander, Vikings,CB Strike (and most all British dramas) Westworld",Outlander,100
63,4,"History drama-Vikings,Kid friendly-Casper,Sometimes the conversations while watching Neon can get serious but we all end up having fun together, :)",Casper,100
62,4,"History drama-Vikings,Kid friendly-Casper,Sometimes the conversations while watching Neon can get serious but we all end up having fun together, :)",Vikings,100
44,3,"Miss scarlet and the duke,knifes out,Dublin murders",Dublin Murders,100


## Further Issues to Address
Another obvious problem is titles that are substrings of other titles.  For instance, 'fear the walking dead' contains the substring 'the walking dead' which is also a content title.  To deal with this we can iterate through content titles for matching in descending order of their string length, when a match of significance (score > 85) is found, we can entirely remove the substring from the response, so smaller length titles cannot be found within it.

Stop words also pose a problem, we could remove all stop words, but I hypothesize leading stop words in a content title will be the most likely for users to forget, rather than internal or ending stop words. ie people may write 'Walking Dead' instead of 'The Walking Dead'.  However, without a larger data set I cannot test this hypothesis, but for the sake of this demonstration I will assume it is true.

## Implementation

In [568]:
try:
    stop_words = stopwords.words('english')
except:
    nltk.download('stopwords')
    stop_words = stopwords.words('english')

def remove_leading_stop_word(content_title):
    global stop_words
    for stop_word in stop_words:
        found_index = content_title.find(stop_word+' ')
        if found_index == 0:
            return content_title[len(stop_word)+1:]
    return content_title

### New Content Title Transformations:

In [569]:
content_titles_df['transformed'] = content_titles_df.apply(lambda x: remove_leading_stop_word(x['transformed']), axis=1)
content_titles_df['content_length'] = content_titles_df.apply(lambda x: len(x['transformed']), axis=1)
content_titles_df = content_titles_df.sort_values(by = ['content_length'], ascending=False)
content_titles_df = content_titles_df.drop(['content_length'], axis=1)
content_titles_df.head()

,Content_name,transformed
8,Miss Scarlet and the Duke,miss scarlet and the duke
0,Fear the Walking Dead,fear the walking dead
7,Spies in Disguise,spies in disguise
5,Gangs of London,gangs of london
14,Game of Thrones,game of thrones


In [570]:
def find_next_space_2(start_index : int, minimum_search_distance : int, text : str):
    check_index = start_index+minimum_search_distance
    space_index = -1
    alphas_observed = 0
    for i in range(start_index, len(text)):
        if text[i] == '|':
            space_index = i
            break
        elif text[i] != ' ':
            alphas_observed+=1
        elif alphas_observed>=minimum_search_distance:
            space_index = i
            break
    #space_index = text[start_index+minimum_search_distance:].find(' ')
    if space_index == -1:
        return len(text)
    else:
        return space_index

def my_fuzzy_match_2(content_title, survey_response : str) -> tuple:
    substring_left_index = 0
    title_len = len(content_title.replace(' ' , ''))
    substring_right_index = find_next_space_2(substring_left_index, title_len, survey_response)
    substring = survey_response[substring_left_index : substring_right_index]
    highest_score = 0
    best_match_indices = (-1,-1)
    while True:
        new_score = fuzz.ratio(substring.replace(' ','').replace('|',''), content_title.replace(' ','').replace('|',''))
        if new_score > highest_score:
            highest_score = new_score
            best_match_indices = (substring_left_index, substring_right_index)
        highest_score = max(highest_score, new_score)
        if highest_score == 100:
            break
        substring_left_index = find_next_space_2(substring_left_index, 1, survey_response)+1
        if substring_left_index >= len(survey_response)-1:
            break
        substring_right_index = find_next_space_2(substring_left_index, title_len, survey_response)
        substring = survey_response[substring_left_index : substring_right_index]
    return (highest_score, best_match_indices)

### New loop to produce scores and remove sub-strings where matches are found:

In [571]:
customer_id_column = []
response_column = []
title_column = []
score_column = []
match_threshold = 85
for index_2, row_2 in survey_response_df.iterrows():
    transformed_response = row_2['transformed']
    for index_1, row_1 in content_titles_df.iterrows():
        customer_id_column.append(row_2['Customer_id'])
        response_column.append(row_2['Response'])
        title_column.append(row_1['Content_name'])
        results = my_fuzzy_match_2(row_1['transformed'], transformed_response)
        if results[0] > match_threshold:
            transformed_response = transformed_response[:max(results[1][0]-1,0)] +'|' + transformed_response[results[1][1]+1:]
        score_column.append(results[0])
result_dict = {
    'Customer_id':customer_id_column,
    'Response':response_column,
    'Title':title_column,
    'Score': score_column}
result_df = pd.DataFrame(result_dict).sort_values(by=['Score'], ascending=False)
#result_df.head(50)

In [572]:
df_styler =result_df.style.set_properties(**{'text-align': 'left'})
df_styler.set_table_styles([dict(selector='th', props=[('text-align', 'left')])])

,Customer_id,Response,Title,Score
72,5,"The Undoing,Game of thrones,Outlander, Vikings,CB Strike (and most all British dramas) Westworld",Game of Thrones,100
10,1,"Fear the walking dead,Supernatural (huge fan and sad it has finished),The Gentlemen, Outlander",The Gentlemen,100
1,1,"Fear the walking dead,Supernatural (huge fan and sad it has finished),The Gentlemen, Outlander",Fear the Walking Dead,100
20,2,A lot! -good doctor -gangs of London - the gentleman -ma -spies in disguise,Gangs of London,100
64,4,"History drama-Vikings,Kid friendly-Casper,Sometimes the conversations while watching Neon can get serious but we all end up having fun together, :)",Vikings,100
34,3,"Miss scarlet and the duke,knifes out,Dublin murders",Miss Scarlet and the Duke,100
66,4,"History drama-Vikings,Kid friendly-Casper,Sometimes the conversations while watching Neon can get serious but we all end up having fun together, :)",Casper,100
24,2,A lot! -good doctor -gangs of London - the gentleman -ma -spies in disguise,The Good Doctor,100
11,1,"Fear the walking dead,Supernatural (huge fan and sad it has finished),The Gentlemen, Outlander",Outlander,100
39,3,"Miss scarlet and the duke,knifes out,Dublin murders",Dublin Murders,100


## The problems I won't Address
There are still many potential pitfalls with this method, albeit, problems that should present less often.  Some of these are:
 - Responses with no punctuation.  Example: 'ireallylikedthewalkingdead', my white space splitting method would fail to match on a string like this.  A method that generates whitespace here would likely work well, ie reconstruct a sentence from this response so it reads 'I really liked the walking dead', this would then work with the method above.
 - Responses with two subsequent titles, that when joined in the middle produce a new title.  Rather than the two titles being found, the central one could be found which is then removed, stopping the two from being found.  A fix for this is fairly straight forward, but I feel goes beyond the scope of this exercise.
 - Descriptive language that causes a title to be transformed into a longer title, resulting in the substring it was found in being removed and then subsequently not found for the correct title.  Example: 'I fear The Walking Dead' < the person is frightened by the walking dead, but my method would match 'Fear the Walking Dead' to this, resulting in 'The Walking Dead' not being found.  The only solution to this problem that I can see is implementing more complex NLP methods such as POS tagging, NER, etc that provide context to the words found in the sentences.
 
 This brings me to the final solution I've implemented for this problem.  We can construct a much more robust solution using ML techniques.